In [1]:
from mysql_connection import get_cursor
import time
import pandas as pd
import numpy as np

cursor = get_cursor()
# Start the timer
start = time.perf_counter()

## Call get food checks by id with the user id

In [2]:
user_id = (2,)
foodchecks = cursor.callproc("GetFoodChecksByID", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


## Parse all the results, and create the WHERE clause

In [3]:
column_rows = ["IsVegetarian", "IsVegan", "IsHalal", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy"]

def print_food_checks(result) :
    for i in range(len(result)):
        print(f"{column_rows[i]} : {result[i]}")

print_food_checks(checks)

def food_checks_builder(result):
    food_checks = []
    for i in range(len(result)):
        if(result[i]):
            food_checks.append(f"{column_rows[i]}=1")
    return " AND ".join(food_checks)

print("---------------------------------")
print(food_checks_builder(checks))

IsVegetarian : 0
IsVegan : 0
IsHalal : 0
IsKosher : 0
HasLactose : 1
HasNuts : 1
HasGluten : 1
HasEgg : 1
HasSoy : 1
---------------------------------
HasLactose=1 AND HasNuts=1 AND HasGluten=1 AND HasEgg=1 AND HasSoy=1


## Get the user's most liked and disliked items


In [4]:
cursor.execute(f"SELECT FoodTagID FROM FoodOpinionRightSwipePercent WHERE UserID={user_id[0]};")
result = cursor.fetchall()
food_tags = [str(i[0]) for i in result[:5] + result[-5:]] # Select top 5 and bottom 5 rated tags
print(food_tags)

['2', '4', '1', '28', '27', '9', '8', '7', '6', '5']


## Get all the user id's with at least the food checks that you have
### i.e. You are vegetarian... Some one who is vegetarian, egg-free, and lactose-free will also be selected 

In [5]:
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID IN(SELECT UserID FROM tat.UserFoodCheckView WHERE {food_checks_builder(checks)} AND UserID<>{user_id[0]}) AND FoodTagID IN ({','.join(food_tags)});")
result = cursor.fetchall()
print(pd.DataFrame(result))

     0   1       2
0    6   6  1.8333
1    6   7  1.7895
2    6   8  1.7895
3    6   5  1.6667
4    6   1  1.3333
5    6   9  1.1642
6    6  27  0.4444
7    6   2  0.2857
8    6   4  0.2222
9    6  28  0.1429
10   7   2  2.0000
11   7  27  1.8889
12   7   1  1.8519
13   7   4  1.7778
14   7  28  1.4667
15   7   9  1.3846
16   7   6  0.8889
17   7   7  0.6667
18   7   8  0.6667
19   7   5  0.2222
20   8   2  1.9333
21   8   4  1.6500
22   8   1  0.7333
23   8   9  0.2385
24   8  27  0.2000
25   8  28  0.0930
26   8   7  0.0000
27   8   6  0.0000
28   8   5  0.0000
29   8   8  0.0000
30   9   5  1.7333
31   9   7  1.6667
32   9   8  1.6667
33   9   6  1.6471
34   9   1  1.1111
35   9   4  0.7647
36   9  27  0.7500
37   9   2  0.6400
38   9  28  0.5294
39   9   9  0.4878
40  11   5  1.4000
41  11   7  1.0667
42  11   8  1.0667
43  11   6  1.0000
44  11   1  0.2667
45  11   9  0.2041
46  11   4  0.2000
47  11  28  0.1600
48  11   2  0.1333
49  11  27  0.0000


In [6]:
tag_dict = {}
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID={user_id[0]};")
user = cursor.fetchall()
for row in user:
    tag_dict[row[1]] = float(row[2])

print(tag_dict)

{2: 2.0, 4: 2.0, 1: 1.0, 28: 0.0, 27: 0.0, 26: 0.0, 25: 0.0, 24: 0.0, 23: 0.0, 22: 0.0, 21: 0.0, 20: 0.0, 18: 0.0, 17: 0.0, 16: 0.0, 15: 0.0, 14: 0.0, 13: 0.0, 12: 0.0, 11: 0.0, 9: 0.0, 8: 0.0, 7: 0.0, 6: 0.0, 5: 0.0}


In [8]:
ptr = 0
count_dict = {55: 2}
TOLERANCE = 0.3

# Go through all results, and count how many tags are within the tolerance
for row in result:
    user_val = tag_dict[row[1]]
    # Within the user's swipe percentage + / - 0.3
    if user_val - TOLERANCE <= row[2] <= user_val + TOLERANCE:
        try:
            count = count_dict[row[0]]
            count_dict[row[0]] = count + 1
        except:
            count_dict[row[0]] = 1

ids_to_get = []

print(count_dict)
for user in count_dict.items():
    if(user[1] >= 5):
        ids_to_get.append(str(user[0]))

print(ids_to_get)

{55: 2, 6: 1, 7: 3, 8: 9, 9: 1, 11: 3}
['8']


In [9]:
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID IN ({','.join(ids_to_get)});")
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ["UserID", "FoodTagID", "RightSwipePct"]
similarities = []
for row in df.iterrows():
    id = row[1][0]
    similarities.append(count_dict[id] / 5)

df["Bias"] = similarities
print(df)

    UserID  FoodTagID RightSwipePct  Bias
0        8          2        1.9333   1.8
1        8          4        1.6500   1.8
2        8         32        1.6400   1.8
3        8         36        0.8000   1.8
4        8          1        0.7333   1.8
5        8         21        0.5610   1.8
6        8         20        0.5000   1.8
7        8         34        0.4000   1.8
8        8         33        0.4000   1.8
9        8         18        0.4000   1.8
10       8          9        0.2385   1.8
11       8         27        0.2000   1.8
12       8         28        0.0930   1.8
13       8         25        0.0000   1.8
14       8         22        0.0000   1.8
15       8         26        0.0000   1.8
16       8         29        0.0000   1.8
17       8         30        0.0000   1.8
18       8         31        0.0000   1.8
19       8          5        0.0000   1.8
20       8         35        0.0000   1.8
21       8         24        0.0000   1.8
22       8         23        0.000

## We've got the data, now time to shove it into a user array

In [10]:
from user import User
user_data = []
for user_id in ids_to_get:
    # For each user id in the one's we want to get
    # Extract that part of the dataframe, and create a new user object from it
    user_data.append(User(df[df["UserID"] == int(user_id)]))


In [11]:


end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")



Done in 309.1228 seconds


In [12]:
 for user in user_data:
     user.print()

ID: 8 	 Bias: 1.8 
Food Tags: [[2, Decimal('1.9333')], [4, Decimal('1.6500')], [32, Decimal('1.6400')], [36, Decimal('0.8000')], [1, Decimal('0.7333')], [21, Decimal('0.5610')], [20, Decimal('0.5000')], [34, Decimal('0.4000')], [33, Decimal('0.4000')], [18, Decimal('0.4000')], [9, Decimal('0.2385')], [27, Decimal('0.2000')], [28, Decimal('0.0930')], [25, Decimal('0.0000')], [22, Decimal('0.0000')], [26, Decimal('0.0000')], [29, Decimal('0.0000')], [30, Decimal('0.0000')], [31, Decimal('0.0000')], [5, Decimal('0.0000')], [35, Decimal('0.0000')], [24, Decimal('0.0000')], [23, Decimal('0.0000')], [6, Decimal('0.0000')], [17, Decimal('0.0000')], [16, Decimal('0.0000')], [15, Decimal('0.0000')], [14, Decimal('0.0000')], [13, Decimal('0.0000')], [12, Decimal('0.0000')], [11, Decimal('0.0000')], [8, Decimal('0.0000')], [7, Decimal('0.0000')]]
